In [2]:
!pip install kaggle
!mkdir .kaggle

import json
token = {"username":"......","key":"......."}
with open('/content/.kaggle/kaggle.json', 'w') as file:
    json.dump(token, file)

!cp /content/.kaggle/kaggle.json ~/.kaggle/kaggle.json
!kaggle config set -n path -v{/content}
!chmod 600 /root/.kaggle/kaggle.json
print('Скачиваю датасеты')
!kaggle competitions download -c sf-dl-movie-genre-classification -p /content
!ls

mkdir: cannot create directory ‘.kaggle’: File exists
- path is now set to: {/content}
Скачиваю датасеты
  0% 0.00/718k [00:00<?, ?B/s]
100% 718k/718k [00:00<00:00, 47.4MB/s]
 36% 5.00M/13.9M [00:00<00:00, 21.3MB/s]
100% 13.9M/13.9M [00:00<00:00, 46.5MB/s]
 36% 5.00M/13.8M [00:00<00:00, 34.6MB/s]
100% 13.8M/13.8M [00:00<00:00, 54.8MB/s]
 37% 5.00M/13.7M [00:00<00:00, 20.1MB/s]
100% 13.7M/13.7M [00:00<00:00, 45.6MB/s]
 65% 9.00M/13.8M [00:00<00:00, 22.3MB/s]
100% 13.8M/13.8M [00:00<00:00, 31.1MB/s]
sample_data	       test.csv.zip	  train.csv.zip
sample-submission.csv  test_data.txt.zip  train_data.txt.zip


In [0]:
import random
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model
import tensorflow_hub as hub

In [0]:
train = pd.read_csv('train.csv.zip')

In [0]:
test = pd.read_csv('test.csv.zip')

In [44]:
len(train['text'])

54214

In [45]:
classes =  list(set(train.genre))
classes

['history',
 'fantasy',
 'adult',
 'romance',
 'game-show',
 'news',
 'war',
 'drama',
 'musical',
 'adventure',
 'crime',
 'thriller',
 'western',
 'family',
 'action',
 'documentary',
 'sport',
 'music',
 'animation',
 'biography',
 'reality-tv',
 'short',
 'horror',
 'mystery',
 'sci-fi',
 'talk-show',
 'comedy']

In [7]:
train = train[['text', 'genre']]
train = pd.get_dummies(train, prefix='', prefix_sep='', columns=['genre',])
train.to_csv('train_df.csv', index=False)
train.head(3)

,text,action,adult,adventure,animation,biography,comedy,crime,documentary,drama,family,fantasy,game-show,history,horror,music,musical,mystery,news,reality-tv,romance,sci-fi,short,sport,talk-show,thriller,war,western
0,Listening in to a conversation between his do...,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,A brother and sister with a past incestuous r...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2,As the bus empties the students for their fie...,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [8]:
train.iloc[:3,1:]

,action,adult,adventure,animation,biography,comedy,crime,documentary,drama,family,fantasy,game-show,history,horror,music,musical,mystery,news,reality-tv,romance,sci-fi,short,sport,talk-show,thriller,war,western
0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [0]:
embed = hub.load("https://tfhub.dev/google/tf2-preview/nnlm-en-dim128/1")

In [0]:
embedding = "https://tfhub.dev/google/tf2-preview/nnlm-en-dim128/1"
hub_layer = hub.KerasLayer(embedding, input_shape=[], 
                           dtype=tf.string, trainable=True)

In [31]:
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(128, activation='elu'))
model.add(tf.keras.layers.Dense(128, activation='elu'))
model.add(tf.keras.layers.Dense(27, activation='softmax'))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 128)               124642688 
_________________________________________________________________
dense (Dense)                (None, 128)               16512     
_________________________________________________________________
dense_1 (Dense)              (None, 128)               16512     
_________________________________________________________________
dense_2 (Dense)              (None, 27)                3483      
Total params: 124,679,195
Trainable params: 124,679,195
Non-trainable params: 0
_________________________________________________________________


In [0]:
tf.keras.backend.clear_session()
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [0]:
x_val = train['text'][:5000]
partial_x_train = train['text'][5000:]

y_val = train.iloc[:5000,1:]
partial_y_train = train.iloc[5000:,1:]

In [34]:
history = model.fit(partial_x_train,
                    partial_y_train,
                    epochs=10,
                    batch_size=500,
                    validation_data=(x_val, y_val),
                    verbose=1)

Epoch 1/10
99/99 [==============================] - 5s 49ms/step - loss: 0.8923 - accuracy: 0.7975 - val_loss: 2.1515 - val_accuracy: 0.5332
Epoch 2/10
99/99 [==============================] - 5s 47ms/step - loss: 0.0642 - accuracy: 0.9971 - val_loss: 2.7813 - val_accuracy: 0.5256
Epoch 3/10
99/99 [==============================] - 5s 48ms/step - loss: 0.0151 - accuracy: 0.9992 - val_loss: 3.1131 - val_accuracy: 0.5208
Epoch 4/10
99/99 [==============================] - 5s 47ms/step - loss: 0.0081 - accuracy: 0.9992 - val_loss: 3.3380 - val_accuracy: 0.5210
Epoch 5/10
99/99 [==============================] - 5s 49ms/step - loss: 0.0061 - accuracy: 0.9993 - val_loss: 3.5311 - val_accuracy: 0.5208
Epoch 6/10
99/99 [==============================] - 5s 48ms/step - loss: 0.0054 - accuracy: 0.9992 - val_loss: 3.6729 - val_accuracy: 0.5158
Epoch 7/10
99/99 [==============================] - 5s 48ms/step - loss: 0.0042 - accuracy: 0.9992 - val_loss: 3.7510 - val_accuracy: 0.5154
Epoch 8/10
99

In [0]:
pr = model.predict(test.text.values)

In [39]:
len(np.argmax(pr, axis=-1))

54200

In [40]:
len(test)

54200